In [1]:
import os 

In [2]:
%pwd
os.chdir('../')
%pwd

'c:\\Users\\tchok\\OneDrive\\Bureau\\My_github\\clustering-insured-population'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PreparBaseModelConfig:
    """
    Base class for all model configurations.
    """
    root_dir : Path 
    base_model_path : Path
    params_n_clusters : int

In [4]:
from insuredSegmenter.constants import *
from insuredSegmenter.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH
        ):

        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))

        create_directories([self.config.artifacts_root])
    
    def get_base_model_config(self) -> PreparBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config = PreparBaseModelConfig( 
              root_dir = Path(config.root_dir),
              base_model_path = Path(config.base_model_path),
              params_n_clusters = int(self.params.Kmeans.n_clusters),
              
        )
        return prepare_base_model_config  

In [5]:
from sklearn.cluster import KMeans
import joblib
from typing import Any
import os 
from insuredSegmenter import logger

In [ ]:

class PreparBaseModel:
    def __init__(self, config: PreparBaseModelConfig):
        self.config = config
        self.model = None
    
    def get_base_model(self) -> KMeans:
        """
        Create and save a KMeans base model based on configuration.
        
        Returns:
            KMeans: Initialized KMeans model
        """
        self.model = KMeans(
            n_clusters=self.config.params_n_clusters,
            random_state=42,
            # Add other necessary KMeans parameters here
        )
        
        # Save the model
        self.save_base_model()
        return self.model
    
    def load_base_model(self) -> Any:
        """
        Load a pre-saved base model from the specified path.
        
        Returns:
            Any: Loaded model
        """
        try:
            logger.info(f"Loading base model from {self.config.base_model_path}")
            self.model = joblib.load(str(self.config.base_model_path))
            logger.info("Base model loaded successfully")
            return self.model
        except FileNotFoundError as e:
            logger.error(f"Model file not found: {e}")
            raise
    
    def save_base_model(self) -> None:
        """
        Save the current model to the specified path.
        """
        if self.model is None:
            raise ValueError("No model to save. Call get_base_model() first.")
        
        if not self.config.base_model_path:
            
            raise ValueError("Base model path is not specified")
        
        try:
            logger.info(f"Saving base model to {self.config.base_model_path}"),
            joblib.dump(self.model, str(self.config.base_model_path)),
            logger.info(f"Base model saved successfully to {self.config.base_model_path}"),
        except Exception as e:
            logger.exception(f"Error saving base model to {self.config.base_model_path}: {e}"),
            raise

In [7]:
try : 
    config = ConfigurationManager()
    base_model_config = config.get_base_model_config()
    base_model = PreparBaseModel(config=base_model_config)
    base_model.get_base_model()
except Exception as e:
    raise e


[2025-05-12 19:17:39,198: INFO: common: YAML file loaded successfully: C:\Users\tchok\OneDrive\Bureau\My_github\clustering-insured-population\config\config.yaml]
[2025-05-12 19:17:39,214: INFO: common: YAML file loaded successfully: C:\Users\tchok\OneDrive\Bureau\My_github\clustering-insured-population\params.yaml]
[2025-05-12 19:17:39,220: INFO: common: created directory at: artifacts]
[2025-05-12 19:17:39,226: INFO: common: created directory at: artifacts/prepare_base_model]
artifacts\prepare_base_model\base_model.pkl
[2025-05-12 19:17:39,230: INFO: 3839695428: Saving base model to artifacts\prepare_base_model\base_model.pkl]
[2025-05-12 19:17:39,234: INFO: 3839695428: Base model saved successfully to artifacts\prepare_base_model\base_model.pkl]
